<a href="https://colab.research.google.com/github/claudialeguiza/AA1-TUIA-Kidonakis-Leguiza/blob/navegador/predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import RobustScaler, FunctionTransformer
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
import keras
from keras.models import Sequential, load_model
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from keras.metrics import Precision
from keras.utils import to_categorical
import joblib
from datetime import date
import warnings
warnings.simplefilter('ignore')

In [128]:
df = pd.read_csv('/content/weatherAUS.csv', delimiter = ",")

In [129]:
def preprocesamiento(datos):

    data = datos[datos.Location\
                      .isin(( 'Sydney','SydneyAirport','Melbourne', 'MelbourneAirport',\
                             'Canberra','Adelaide', 'MountGambier','Cobar', 'Dartmoor' ))]


    data = data.drop('Unnamed: 0', axis =1)

    # Definir columnas con valores nulos
    columnas_con_nulos = ['MinTemp', 'MaxTemp', 'Rainfall', 'Evaporation', 'Sunshine',
                          'WindGustSpeed', 'WindSpeed9am', 'WindSpeed3pm','Humidity9am',
                          'Humidity3pm', 'Pressure9am','Pressure3pm', 'Cloud9am',
                          'Cloud3pm', 'Temp9am', 'Temp3pm', 'RainfallTomorrow']

    # Rellenar valores faltantes en 'RainToday' y 'RainTomorrow'
    data['RainToday'] = data.groupby('Date')['RainToday'].transform(lambda x: x.fillna(x.mode().iloc[0]))
    data['RainTomorrow'] = data.groupby('Date')['RainTomorrow'].transform(lambda x: x.fillna(x.mode().iloc[0]))

    # Rellenar valores faltantes en direcciones del viento
    data['WindGustDir'] = data.groupby('Date')['WindGustDir'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.isna().all() else x)
    data['WindDir9am'] = data.groupby('Date')['WindDir9am'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.isna().all() else x)
    data['WindDir3pm'] = data.groupby('Date')['WindDir3pm'].transform(lambda x: x.fillna(x.mode().iloc[0]) if not x.isna().all() else x)

    # Rellenar valores faltantes con la media por día para las columnas especificadas
    media_por_dia = data.groupby('Date')[columnas_con_nulos].transform('mean')
    data[columnas_con_nulos] = data[columnas_con_nulos].fillna(media_por_dia)

    data['Date'] = pd.to_datetime(data['Date'])

    return data


In [130]:
def crear_columna_season(data):
   data['season'] = data['Date'].apply(asignar_estacion)
   return data

In [131]:
def asignar_estacion(fecha):
    mes = fecha.month
    if mes in [12, 1, 2]:  # Verano: Diciembre, Enero, Febrero
        return 'Summer'
    elif mes in [3, 4, 5]:  # Otoño: Marzo, Abril, Mayo
        return 'Autumn'
    elif mes in [6, 7, 8]:  # Invierno: Junio, Julio, Agosto
        return 'Winter'
    else:  # Primavera: Septiembre, Octubre, Noviembre
        return 'Spring'

In [132]:
def codificar_variables(data):
    data1 = pd.get_dummies(data, columns=['RainToday', 'RainTomorrow','season', 'Location'], drop_first=True)

    # Crear columnas para WindGustDir, WindDir9am, WindDir3pm
    wind_directions = ["SW", "S", 'SSW', 'W', 'SSE', 'E', 'SE', 'NE', 'NNE', 'WSW', 'WNW', 'NW', 'N', 'ESE', 'ENE']
    for var in wind_directions:
        data1[f'WindGustDir_{var}'] = (data['WindGustDir'] == var).astype(int)
        data1[f'WindDir9am_{var}'] = (data['WindDir9am'] == var).astype(int)
        data1[f'WindDir3pm_{var}'] = (data['WindDir3pm'] == var).astype(int)

    return data1.drop(columns=['WindGustDir', 'WindDir9am', 'WindDir3pm'])

In [133]:
def robust_df(data):
  scaler = RobustScaler()
  data_scaled = scaler.fit_transform(data)
  return data_scaled

In [134]:
def truncar_dividir_df(data):
    data = data.sort_values(["Date"])
    fecha_especifica = '2009-01-01'
    data_filtrada = data[data['Date'] >= fecha_especifica]

    data_filtrada.reset_index(drop=True, inplace=True)  # Resetea el índice y no crea uno nuevo
    data_train = data_filtrada.iloc[:21658]


    return data_train

In [135]:
def eliminar_columnas_estandarizar(data):
    # Separar variables independientes y dependientes
    X_regresion = data.drop(columns =['RainfallTomorrow','Date'])
    X_scaled = robust_df(X_regresion)
    y_regresion = data['RainfallTomorrow']
    y_scaled = robust_df(y_regresion.values.reshape(-1,1))
    return X_scaled, y_scaled

In [136]:
def estandarizar_balancear_clas(data):
    X_clasificacion = data.drop(columns=['RainTomorrow_Yes','Date','RainfallTomorrow'])
    X_scaled1 = robust_df(X_clasificacion)
    y_clasificacion = data['RainTomorrow_Yes']
    y_scaled1 =robust_df(y_clasificacion.values.reshape(-1,1))
    smote = SMOTE(random_state=42)
    X_smote_scaled, y_smote_scaled = smote.fit_resample(X_scaled1, y_scaled1)

    return X_smote_scaled, y_smote_scaled

In [137]:
def cargar_modelo_regresion():
    # Cargamos el modelo
      modelo_regresion = load_model('/content/regression_model.h5')
      return modelo_regresion

In [138]:
def cargar_modelo_clasificacion():
      modelo_clasif = load_model('/content/classification_model_optimized.h5')
      return modelo_clasif

In [139]:
pipeline_prepara_datos = Pipeline([
    ('preproceso', FunctionTransformer(preprocesamiento, validate=False)),
    ('season', FunctionTransformer(crear_columna_season, validate=False)),
    ('codificar', FunctionTransformer(codificar_variables, validate=False))
])

# Obtener datos de entrenamiento
df_procesado = pipeline_prepara_datos.fit_transform(df)

pipeline_train_split = Pipeline([
    ('split', FunctionTransformer(truncar_dividir_df, validate=False)),
    ('estandarizar', FunctionTransformer(eliminar_columnas_estandarizar, validate=False)),
    ])

# Obtener datos de entrenamiento
X_train_scaled, y_train_scaled = pipeline_train_split.fit_transform(df_procesado)

regression_model = cargar_modelo_regresion()

# Entrenar el modelo
regression_model.fit(X_train_scaled, y_train_scaled, epochs=100, batch_size=32)

#joblib.dump(pipeline_modelo_regresion, 'regresion_pipeline.joblib')

Epoch 1/100
677/677 [==============================] - 2s 2ms/step - loss: 37.4688 - mae: 2.7569
Epoch 2/100
677/677 [==============================] - 1s 2ms/step - loss: 31.8416 - mae: 2.6164
Epoch 3/100
677/677 [==============================] - 1s 2ms/step - loss: 29.7078 - mae: 2.3909
Epoch 4/100
677/677 [==============================] - 1s 2ms/step - loss: 27.5030 - mae: 2.2423
Epoch 5/100
677/677 [==============================] - 2s 3ms/step - loss: 26.6835 - mae: 2.0997
Epoch 6/100
677/677 [==============================] - 2s 3ms/step - loss: 24.9715 - mae: 1.9669
Epoch 7/100
677/677 [==============================] - 1s 2ms/step - loss: 24.0115 - mae: 1.8841
Epoch 8/100
677/677 [==============================] - 1s 2ms/step - loss: 24.5244 - mae: 1.8315
Epoch 9/100
677/677 [==============================] - 1s 2ms/step - loss: 24.7723 - mae: 1.8071
Epoch 10/100
677/677 [==============================] - 1s 2ms/step - loss: 23.6536 - mae: 1.7411
Epoch 11/100
677/677 [=======

In [140]:
pipeline_train_split_clas = Pipeline([
    ('split', FunctionTransformer(truncar_dividir_df, validate=False)),
    ('estandarizar_clas', FunctionTransformer(estandarizar_balancear_clas, validate=False)),
    ])

classification_model = cargar_modelo_clasificacion()

# Obtener datos de entrenamiento
X_smote, y_smote = pipeline_train_split_clas.fit_transform(df_procesado)

# Entrenar el modelo
classification_model.fit(X_smote, y_smote, epochs=100, batch_size=16)


#joblib.dump(pipeline_modelo_clasificacion, 'clasificacion_pipeline.joblib')

Epoch 1/100
2078/2078 [==============================] - 5s 2ms/step - loss: 0.4556 - precision: 0.8091
Epoch 2/100
2078/2078 [==============================] - 3s 2ms/step - loss: 0.4351 - precision: 0.7987
Epoch 3/100
2078/2078 [==============================] - 3s 2ms/step - loss: 0.4272 - precision: 0.8024
Epoch 4/100
2078/2078 [==============================] - 3s 2ms/step - loss: 0.4217 - precision: 0.8032
Epoch 5/100
2078/2078 [==============================] - 4s 2ms/step - loss: 0.4173 - precision: 0.8053
Epoch 6/100
2078/2078 [==============================] - 3s 2ms/step - loss: 0.4138 - precision: 0.8057
Epoch 7/100
2078/2078 [==============================] - 3s 2ms/step - loss: 0.4107 - precision: 0.8084
Epoch 8/100
2078/2078 [==============================] - 4s 2ms/step - loss: 0.4081 - precision: 0.8097
Epoch 9/100
2078/2078 [==============================] - 4s 2ms/step - loss: 0.4057 - precision: 0.8122
Epoch 10/100
2078/2078 [==============================] - 4s 2ms

In [141]:
def mostrar_ciudades():
    print("Las ciudades sobre las cuales puede hacer consulta son las siguientes:","\n")
    print("1. Sidney")
    print("2. SidneyAirport")
    print("3. Melbourne")
    print("4. MelbournAirport")
    print("5. Canberra")
    print("6. Adelaide")
    print("7. MountGambier")
    print("8. Cobar")
    print("9. Dartmoor")
    print("10. Salir")
    return

In [142]:
def validar_ciudad(opcion):

    if opcion == "1":
        location = "Sidney"
    elif opcion == "2":
        location = "SidneyAirport"
    elif opcion == "3":
        location = "Melbourne"
    elif opcion == "4":
        location = "MelbourneAirport"
    elif opcion == "5":
       location = "Canberra"
    elif opcion == "6":
       location = "Adelaida"
    elif opcion == "7":
       location = "MontGambier"
    elif opcion == "8":
       location = "Cobar"
    elif opcion == "9":
       location = "Dartmoor"
    elif opcion == "10":
        print("Gracias por usar el predictor")
        return

    return location

In [143]:
def validar_temp(temp):
	try:
		decimal = float(temp)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= -10 and decimal <= 50)

In [144]:
def validar_presion(pres):
	try:
		decimal = float(pres)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 950 and decimal <= 1041)

In [145]:
def validar_nubosidad(nub):
	try:
		decimal = float(nub)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 0 and decimal <= 10)

In [146]:
def validar_humedad(humed):
	try:
		decimal = float(humed)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 0 and decimal <= 100)

In [147]:
def validar_otro(v):
	try:
		decimal = float(v)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 0 and decimal <= 150)

In [148]:
def validar_sol(horas):
	try:
		decimal = float(horas)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 0 and decimal <= 24)

In [149]:
def validar_velocidad(km):
	try:
		decimal = float(km)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return (decimal >= 0 and decimal <= 300)

In [150]:
def mostrar_opciones():
    print("Opciones direccion del viento permitidas:","\n")
    print("N - Norte   NE - Noreste             NW - Noroeste        NNE - Norte Noreste", "\n" )
    print("S - Sur     SE - Sureste             SW - Suroeste        SSE - Sur Sureste" , "\n")
    print("E - Este    ESE - Este Sureste       ENE - Este Noreste   SSW - Sur Suroeste", "\n" )
    print("W - Oeste   WSW - Oeste Suroeste     WNW - Oeste Noroeste", "\n" )

    return

In [151]:
def obtener_datos_clima(dato):

    lista_direcciones = ["N", "NE", "NW", "NNE", "S", "SE", "SW", "SSE", "E",\
                           "ESE", "ENE", "W", "WSW", "WNW", "SSW"]

    if dato == 'MinTemp':
       valor = input("Temperatura minima(grados C):")

       while (validar_temp(valor) == False):
           print("Temperatura no valida")
           valor = input("Temperatura minima (grados C):")
       return float(valor)

    if dato == 'MaxTemp':
       valor = input("Temperatura maxima(grados C):")
       while (validar_temp(valor) == False):
           print("Temperatura no valida")
           valor = input("Temperatura maxima (grados C):")
       return float(valor)

    if dato == 'Rainfall':
       valor = input("Lluvia caida (mm) - Ingrese 0 si no llovio:")
       while (validar_otro(valor) == False) :
           print("Valor no valido")
           valor = input("Milimetros de lluvia caida(Ingrese 0 si no llovio):")
       return float(valor)

    if dato == 'Evaporation':
       valor = input("Evaporacion (mm):")
       while (validar_otro(valor) == False):
            print("Valor no valido")
            valor = input("Evaporacion (mm):")
       return float(valor)

    if dato == 'Sunshine':
       valor = input("Horas de Sol:")
       while  (validar_sol(valor) == False):
           print("Valor no valido")
           valor = input("Horas de Sol:")
       return float(valor)

    if dato == 'WindGustDir':
       valor = input("Direccion rafagas de viento fuerte (ultimas 24 hs):")
       while valor not in lista_direcciones:
             print("Direccion no valida")
             valor = input("Direccion rafagas de viento fuerte (ultimas 24 hs):")
       return valor

    if dato == 'WindGustSpeed':
       valor = input("Velocidad de las rafagas de viento fuerte(ultimas 24 hs - km/h):")
       while (validar_velocidad(valor) == False):
              print("Valor no valido")
              valor = input("Velocidad de las rafagas de viento fuerte(ultimas 24 hs - km/h):")
       return float(valor)

    if dato == 'WindDir9am':
       valor = input("Direccion del viento 9AM: ")
       while valor not in lista_direcciones:
              print("Direccion no valda")
              valor = input("Direccion del viento 9AM: ")
       return valor

    if dato == 'WindDir3pm':
       valor = input("Direccion del viento 3PM: ")
       if valor == '':
          valor = None
          return valor
       else:
          while valor not in lista_direcciones:
             print("direccion no valida")
             valor = input("Direccion del viento 3PM: ")
          return valor

    if dato == 'WindSpeed9am':
       valor = input("Velocidad del viento 9AM (km/h): ")
       while (validar_velocidad(valor)== False) :
             print("Valor no valido")
             valor = input("Velocidad del viento 9AM(km/h):")
       return float(valor)

    if dato == 'WindSpeed3pm':
       valor = input("Velocidad del viento 3PM (km/h):")
       if valor == '':
          valor = np.NaN
          return valor
       else:
         while (validar_velocidad(valor)== False):
             print("Valor no valido")
             valor = input("Velocidad del viento 3PM(km/h):")
         return float(valor)

    if dato =='Humidity9am':
       valor = input('Humedad 9AM: ')
       while validar_humedad(valor) == False:
             print('Humedad no valida')
             valor = input('Humedad 9AM:')
       return float(valor)

    if dato == 'Humidity3pm':
       valor = input("Humedad 3PM:")
       if valor == '':
          valor = np.NaN
          return valor
       else:
         while (validar_humedad(valor) == False):
           print('Humedad no valida')
           valor = input('Humedad 3PM: ')
         return float(valor)

    if dato == 'Pressure9am':
       valor = input("Presion 9AM:")
       while (validar_presion(valor) == False):
             print("Presion no valida")
             valor = input("presion 9 AM:")
       return float(valor)

    if dato == 'Pressure3pm':
       valor = input("Presion 3PM: ")
       if valor == '':
          valor = np.NaN
          return valor
       else:
          while (validar_presion(valor) == False):
             print("Presion no valida")
             valor = input("Presion 3PM:")
          return float(valor)

    if dato == 'Cloud9am':
       valor = input("Nubosidad 9AM:")
       while (validar_nubosidad(valor) == False):
             print("Nubosidad no valida")
             valor = input("Nubosidad 9AM:")
       return float(valor)

    if dato == 'Cloud3pm':
       valor = input("Nubosidad 3PM:")
       if valor == '':
          valor = np.NaN
          return valor
       else:
          while (validar_nubosidad(valor) == False):
             print("Nubosidad no valida")
             validar_otras = input("Nubosidad 3PM:")
          return float(valor)

    if dato =='Temp9am':
       valor = input("Temperatura 9AM:")
       while (validar_temp(valor) == False):
             print("Temperatura no valida")
             valor = input("Temperatura 9AM: ")
       return float(valor)

    if dato == 'Temp3pm':
       valor = input("Temperatura 3PM: ")
       if valor == '':
          valor = np.NaN
          return valor
       else:
         while (validar_temp(valor) == False):
             print("Temperatura no valida")
             valor = input("Temperatura 3PM: ")
         return float(valor)

    if dato == 'RainToday':
       valor = input("Llueve hoy?:(Yes/No)")
       while (valor != 'Yes' and valor != 'No'):
           print("Respuesta no valida")
           valor = input("Llueve hoy?:(Yes/No)")
       return valor


In [152]:
def validar_opcion(valor):
	try:
		decimal = float(valor)
	except ValueError:
		return False #si la conversión falla devolvemos False

	return decimal >= 0 and decimal <= 10

In [158]:
def interactuar_con_usuario():
    print("¡Bienvenido al predictor del clima de Australia para mañana!", "\n")

    armar_prediccion = {}
    fecha = date.today()
    mostrar_ciudades()
    opcion = input("Seleccione una opcion: ")
    while validar_opcion(opcion) == False:
           print ("opcion no valida")
           opcion = input("Seleccione una opcion: ")
    if opcion == '10':
       print ("Gracias por usar el predictor")
       return

    else:
      print("Ingrese los siguientes datos para predecir", "\n")
      mostrar_opciones()
      armar_prediccion = {'Date': fecha,
                        'Location': validar_ciudad(opcion),
                        'MinTemp': obtener_datos_clima("MinTemp"),
                        'MaxTemp': obtener_datos_clima("MaxTemp"),
                        'Rainfall': obtener_datos_clima("Rainfall"),
                        'Evaporation': obtener_datos_clima("Evaporation"),
                        'Sunshine': obtener_datos_clima("Sunshine"),
                        'WindGustDir': obtener_datos_clima("WindGustDir"),
                        'WindGustSpeed': obtener_datos_clima("WindGustSpeed"),
                        'WindDir9am': obtener_datos_clima("WindDir9am"),
                        'WindDir3pm': obtener_datos_clima("WindDir3pm"),
                        'WindSpeed9am': obtener_datos_clima("WindSpeed9am"),
                        'WindSpeed3pm': obtener_datos_clima("WindSpeed3pm"),
                        'Humidity9am': obtener_datos_clima("Humidity9am"),
                        'Humidity3pm': obtener_datos_clima("Humidity3pm"),
                        'Pressure9am': obtener_datos_clima("Pressure9am"),
                        'Pressure3pm': obtener_datos_clima("Pressure3pm"),
                        'Cloud9am': obtener_datos_clima("Cloud9am"),
                        'Cloud3pm': obtener_datos_clima("Cloud3pm"),
                        'Temp9am': obtener_datos_clima("Temp9am"),
                        'Temp3pm': obtener_datos_clima("Temp3pm"),
                        'RainToday' : obtener_datos_clima("RainToday"),
                          }

      df_prediccion =pd.DataFrame([armar_prediccion])
      return df_prediccion

In [154]:
def preparar_prediccion(df):

    # Crear columnas para WindGustDir, WindDir9am, WindDir3pm
    wind_directions = ["SW", "S", 'SSW', 'W', 'SSE', 'E', 'SE', 'NE', 'NNE', 'WSW', 'WNW', 'NW', 'N', 'ESE', 'ENE']
    for var in wind_directions:
        df[f'WindGustDir_{var}'] = (df['WindGustDir'] == var).astype(int)
        df[f'WindDir9am_{var}'] = (df['WindDir9am'] == var).astype(int)
        df[f'WindDir3pm_{var}'] = (df['WindDir3pm'] == var).astype(int)

    df[f'RainToday_Yes'] = (df['RainToday']== 'Yes').astype(int)

    df['Date'] = pd.to_datetime(df['Date'])
    df['season'] = df['Date'].apply(asignar_estacion)

    # Crear columnas para season
    season_list = ['Spring', 'Summer','Winter' ]
    for season in season_list:
        df[f'season_{season}'] = (df['season']== season).astype(int)

    #Crear columnas para Location
    Location_list= ['Canberra','Cobar', 'Dartmoor','Melbourne','MelbourneAirport',\
                    'MountGambier','Sydney','SydneyAirport']
    for ciudad in  Location_list:
         df[f'Location_{ciudad}'] = (df['Location']== ciudad).astype(int)



    df = df.drop(columns=['WindGustDir', 'WindDir9am', 'WindDir3pm','Date',\
                          'season', 'Location', 'RainToday'])

    return df


In [167]:
df_prediccion = interactuar_con_usuario()
if df_prediccion is not None:
    df_prediccion_filtrada = preparar_prediccion(df_prediccion)
    df_prediccion_filtrada['Prediccion_lluvia'] = classification_model.predict(df_prediccion_filtrada)[0]
    df_prediccion_filtrada['Prediccion_lluvia'] = df_prediccion_filtrada['Prediccion_lluvia'].astype(int)
    df_prediccion_filtrada['Prediccion_mm'] = df_prediccion_filtrada['Prediccion_lluvia'].astype(float)

    if df_prediccion_filtrada['Prediccion_lluvia'][0] == 1:
       print(" Mañana hay un 85% de probabilidad de lluvia")
       df_prediccion_filtrada['Prediccion_mm'] = regression_model.predict(df_prediccion_filtrada)[0]
       print(" Caeran aprox : ", df_prediccion_filtrada['Prediccion_mm'][0], 'mm')
    else:
       print(' Mañana no llovera ')
       df_prediccion_filtrada['Prediccion_mm'] = 0.0
else:
    print("Hasta luego")

¡Bienvenido al predictor del clima de Australia para mañana! 

Las ciudades sobre las cuales puede hacer consulta son las siguientes: 

1. Sidney
2. SidneyAirport
3. Melbourne
4. MelbournAirport
5. Canberra
6. Adelaide
7. MountGambier
8. Cobar
9. Dartmoor
10. Salir
Seleccione una opcion: 4
Ingrese los siguientes datos para predecir 

Opciones direccion del viento permitidas: 

N - Norte   NE - Noreste             NW - Noroeste        NNE - Norte Noreste 

S - Sur     SE - Sureste             SW - Suroeste        SSE - Sur Sureste 

E - Este    ESE - Este Sureste       ENE - Este Noreste   SSW - Sur Suroeste 

W - Oeste   WSW - Oeste Suroeste     WNW - Oeste Noroeste 

Temperatura minima(grados C):10
Temperatura maxima(grados C):23
Lluvia caida (mm) - Ingrese 0 si no llovio:0
Evaporacion (mm):6
Horas de Sol:4
Direccion rafagas de viento fuerte (ultimas 24 hs):
Direccion no valida
Direccion rafagas de viento fuerte (ultimas 24 hs):
Direccion no valida
Direccion rafagas de viento fuerte (